# Pass Network

Readapting my pass network with inspiration from Devin Peuler : https://github.com/devinpleuler/analytics-handbook

credits and kudos to him

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mplsoccer import Pitch
%matplotlib inline

In [2]:
with open('C:\\Users\\marwane\\Documents\\Data Science\\events_19769.json',encoding="utf-8") as data:
    raw_events = json.load(data)

In [3]:
import pprint

In [4]:
#Contains the lineup of the 2 teams playing
lineups = raw_events[0:2]
# team lineup
team_lineup = lineups[0]
#team id
team_id = team_lineup['team']['id'] 

In [5]:
starters = {p['player']['id']: {"name":p['player']['name'],"jersey":p['jersey_number']}for p in team_lineup['tactics']['lineup']}

Home team starter

In [6]:
starters

{19419: {'name': 'Marie Hourihan', 'jersey': 1},
 19414: {'name': 'Kirsty Barton', 'jersey': 8},
 16394: {'name': 'Laura Rafferty', 'jersey': 6},
 16395: {'name': 'Victoria Williams', 'jersey': 20},
 16377: {'name': 'Sophie Perry-Campbell', 'jersey': 17},
 16400: {'name': 'Kayleigh Green', 'jersey': 15},
 16397: {'name': 'Chloe Peplow', 'jersey': 12},
 16393: {'name': 'Ellie Brazil', 'jersey': 16},
 16379: {'name': 'Jodie Brett', 'jersey': 18},
 16374: {'name': 'Aileen Whelan', 'jersey': 7},
 16378: {'name': 'Iniabasi Anefiok Umotong', 'jersey': 9}}

Gather all the events by the home team

In [7]:
# all the events by the home team
events = [e for e in raw_events if e['team']['id'] == team_id]

Gather all the passes from the event

In [8]:
passes = [e for e in raw_events if 'pass' in e.keys()]
passes[:5]

[{'id': 'd16bb5e5-50bc-4cdf-b242-93fd05628eb8',
  'index': 5,
  'period': 1,
  'timestamp': '00:00:00.849',
  'minute': 0,
  'second': 0,
  'type': {'id': 30, 'name': 'Pass'},
  'possession': 2,
  'possession_team': {'id': 972, 'name': 'West Ham United LFC'},
  'play_pattern': {'id': 9, 'name': 'From Kick Off'},
  'team': {'id': 972, 'name': 'West Ham United LFC'},
  'player': {'id': 4653, 'name': 'Jane Ross'},
  'position': {'id': 23, 'name': 'Center Forward'},
  'location': [60.0, 40.0],
  'duration': 1.3771,
  'related_events': ['1ea2ce08-d86f-4a21-a341-8e578300ecdd'],
  'pass': {'recipient': {'id': 18148, 'name': 'Lucienne Reichardt'},
   'length': 16.643316,
   'angle': -2.5702553,
   'height': {'id': 1, 'name': 'Ground Pass'},
   'end_location': [46.0, 31.0],
   'body_part': {'id': 40, 'name': 'Right Foot'},
   'type': {'id': 65, 'name': 'Kick Off'}}},
 {'id': 'f426cba8-b0e3-4540-9040-a461321eb268',
  'index': 9,
  'period': 1,
  'timestamp': '00:00:03.467',
  'minute': 0,
  'sec

**Gather the Pass mapping between the players**

In [9]:
""" Uncomment the # to better understand the links"""
matrix = {}
c = 0
for p in passes:
    if 'outcome' not in p['pass'].keys():
        #print(p['index'])
        passer_id = p['player']['id']
        recipient_id = p['pass']['recipient']['id']
        
        a,b = sorted([passer_id,recipient_id])
        
        if a not in matrix.keys():
            matrix[a] = {}
            
            
        #print("\n"+str(c)+" Passer")
        #print(a)
        #print(matrix)   
            
        if b not in matrix[a].keys():
            matrix[a][b] = 0
            
        #print("\n"+"\t"+str(c)+" Recipient")
        #print("\t",b)
        #print("\t"+str(matrix))        
            
        matrix[a][b] += 1
        
        #print("\n"+"\t"+str(c)+" Pass Count")
        #print("\t",matrix[a][b])
        #print("\t"+str(matrix))
                       
     

In [10]:
print(c)

0


In [11]:
# Print 5 first items from the dictionary
list(matrix.items())[:5]

[(4653,
  {18148: 3,
   18156: 8,
   18154: 5,
   18157: 5,
   18159: 5,
   18151: 2,
   22027: 1,
   18153: 3,
   18150: 8,
   18147: 9,
   19896: 2}),
 (18148,
  {18156: 4, 18151: 6, 22027: 5, 18154: 1, 18153: 1, 18150: 5, 18159: 1}),
 (18154, {18157: 3, 18156: 9, 18159: 1, 22027: 1}),
 (16394, {19419: 9, 19414: 23, 16397: 10, 16395: 8, 16400: 8}),
 (16395, {19419: 10, 16397: 11, 19414: 7, 16400: 3, 20034: 2})]

**Gather the average positions of each player of the home team**

In [12]:
positions = {}
for e in events:
    # players from the home team
    if 'player' in e.keys():
        player_id = e['player']['id']
        if player_id not in positions.keys():
            positions[player_id] = {"x":[],"y":[]}
            
        if 'location' in e.keys():
            positions[player_id]['x'].append(e['location'][0])
            positions[player_id]['y'].append(80-e['location'][1])
        
avg_positions = {k:[np.mean(v['x']),np.mean(v['y'])] for k, v in positions.items() if k in starters.keys() }    

In [13]:
avg_positions

{16378: [81.11111111111111, 42.05925925925926],
 16393: [70.45806451612903, 15.935483870967742],
 16400: [67.44221105527639, 27.839195979899497],
 16397: [52.201388888888886, 38.229166666666664],
 16394: [37.258278145695364, 26.867549668874172],
 16374: [67.2751677852349, 57.49664429530201],
 19419: [8.333333333333334, 38.23076923076923],
 16395: [37.99305555555556, 55.4375],
 16379: [73.58904109589041, 48.21917808219178],
 19414: [46.60427807486631, 8.759358288770054],
 16377: [54.29365079365079, 67.0079365079365]}

In [16]:
lines = []
weights = []

for k,v in matrix.items():
    if k in starters.keys():
        origin = avg_positions[k]
        for k_,v_ in matrix[k].items():
            if k_ in starters.keys():
                dest = avg_positions[k_]
                lines.append([*origin,*dest])
                weights.append(v_)

In [20]:
lines[:3]

[[37.258278145695364,
  26.867549668874172,
  8.333333333333334,
  38.23076923076923],
 [37.258278145695364,
  26.867549668874172,
  46.60427807486631,
  8.759358288770054],
 [37.258278145695364,
  26.867549668874172,
  52.201388888888886,
  38.229166666666664]]

In [24]:
weights[:3]

[9, 23, 10]